首先，回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

X = torch.rand(2,20)
net(X)

tensor([[ 0.0821,  0.2362,  0.0857,  0.1586,  0.2144, -0.1131, -0.0810, -0.2366,
          0.3788, -0.0959],
        [-0.1120,  0.2035,  0.1290,  0.1027,  0.2291, -0.0870, -0.0436, -0.0494,
          0.2715, -0.0274]], grad_fn=<AddmmBackward0>)

nn.Sequential定义了一种特殊的Module

自定义块

In [2]:
class MLP(nn.Module): #继承父类Module
    def __init__(self):
        super().__init__() #先调用父类函数
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)
        
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[-0.2758, -0.2190, -0.0473, -0.0328,  0.1136,  0.0747,  0.0558,  0.1934,
          0.2135, -0.1906],
        [-0.2551, -0.2960,  0.0013, -0.0869,  0.1549,  0.1137,  0.0159,  0.2720,
          0.1797, -0.1612]], grad_fn=<AddmmBackward0>)

顺序块

In [4]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for block in args:
            self._modules[block] = block
            
    def forward(self,X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[-0.0376, -0.1467,  0.2258,  0.0997, -0.0317, -0.2873, -0.1108, -0.2849,
          0.2503,  0.0414],
        [ 0.1231, -0.1947,  0.1873,  0.0497, -0.1653, -0.1199, -0.2316, -0.3107,
          0.1547, -0.0952]], grad_fn=<AddmmBackward0>)

在正向传播函数中执行代码

In [5]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad=False)
        self.linear = nn.Linear(20,20)

    def forward(self,X):
        X = self.linear(X)
        X = F.relu(torch.mm(X,self.rand_weight)+1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.1347, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1385, grad_fn=<SumBackward0>)